In [1]:
import os
os.getcwd()

'/Users/alexm/Documents/IE/Machine Learning II/Group assignment'

### Import the required libraries

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

#### Import the trian dataset

In [3]:
data_train = pd.read_csv("modeling_set.csv")

Transform the time_left column from continius to 6 intervals

In [4]:
data_train.time_left = pd.cut(data_train.time_left,bins=[0,50,75,100,125, 150, 175],labels=[1,2,3,4,5,6])

Encode the round_winner column

In [5]:
data_train.round_winner = data_train.round_winner.astype('category').cat.codes

#### Import the validation dataset

In [6]:
data_test = pd.read_csv("validation_set.csv")

Transform the time_left column from continius to 6 intervals

In [7]:
data_test.time_left = pd.cut(data_test.time_left,bins=[0,50,75,100,125, 150, 175],labels=[1,2,3,4,5,6])

In [8]:
categorical_columns = ['map', 'time_left']

In [9]:
columns_to_add = defaultdict(list)
for col in categorical_columns:
    dummies = pd.get_dummies(data_train[col], prefix=col)
    data_train = pd.concat([data_train, dummies], axis=1)
    data_train.drop(col, axis=1, inplace=True)
    columns_to_add[col] += [new_col.replace(col + "_", "") for new_col in dummies.columns]

In [10]:
data_train.head()

,ct_score,t_score,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,...,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo,time_left_1,time_left_2,time_left_3,time_left_4,time_left_5,time_left_6
0,10.0,7.0,False,500.0,500.0,86.0,483.0,14900.0,10400.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,15.0,14.0,False,500.0,500.0,471.0,500.0,22700.0,3500.0,5.0,...,0,0,0,0,0,0,0,1,0,0
2,9.0,13.0,False,500.0,500.0,473.0,0.0,32300.0,21600.0,5.0,...,0,0,0,0,0,0,0,0,0,1
3,14.0,15.0,False,468.0,489.0,495.0,495.0,2850.0,2250.0,5.0,...,0,0,0,0,1,0,0,0,0,0
4,3.0,2.0,False,497.0,400.0,500.0,400.0,5600.0,3050.0,4.0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
data_test = pd.read_csv("validation_set.csv")

In [12]:
data_test.time_left = pd.cut(data_test.time_left,bins=[0,50,75,100,125, 150, 175],labels=[1,2,3,4,5,6])

In [13]:
columns_to_add = defaultdict(list)
for col in categorical_columns:
    dummies = pd.get_dummies(data_test[col], prefix=col)
    data_test = pd.concat([data_test, dummies], axis=1)
    data_test.drop(col, axis=1, inplace=True)
    columns_to_add[col] += [new_col.replace(col + "_", "") for new_col in dummies.columns]

In [14]:
data_test.head()

,ct_score,t_score,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,...,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo,time_left_1,time_left_2,time_left_3,time_left_4,time_left_5,time_left_6
0,2.0,6.0,False,400.0,300.0,400.0,300.0,1100.0,700.0,3.0,...,0,0,0,0,0,0,1,0,0,0
1,5.0,5.0,False,417.0,149.0,495.0,184.0,5550.0,3100.0,3.0,...,1,0,0,0,1,0,0,0,0,0
2,7.0,3.0,False,500.0,500.0,500.0,500.0,5400.0,750.0,4.0,...,0,0,0,0,0,0,0,1,0,0
3,10.0,7.0,False,228.0,300.0,195.0,300.0,800.0,5250.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5.0,4.0,False,400.0,478.0,396.0,497.0,22100.0,1000.0,4.0,...,0,1,0,0,1,0,0,0,0,0


In [29]:
data_test.round_winner = data_test.round_winner.astype('category').cat.codes

In [30]:
test_set = data_test

In [32]:
X_test = test_set.drop("round_winner", axis=1)
y_test = test_set["round_winner"]

Heatmap

In [23]:
X_train = train_set.drop("round_winner", axis=1)
X_test = test_set.drop("round_winner", axis=1)
y_train = train_set["round_winner"]
y_test = test_set["round_winner"]

Random Forrest

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100, max_features = 'auto', max_depth = 20, criterion = 'entropy', min_samples_split = 3, random_state=0)

In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [100, 200, 300],
    'max_features': ['auto'],
    'max_depth' : [10, 20, 30],
    'criterion' :['entropy'], 
    "min_samples_split": [2, 3, 5, 10]
}


In [27]:
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 30,
 'max_features': 'auto',
 'min_samples_split': 2,
 'n_estimators': 300}

In [33]:
from sklearn.metrics import accuracy_score
y_pred = rfc.fit(X_train, y_train).predict(X_test)
print("#Mislabeled points (RF) : {1} out of {0}".format(len(y_test), sum(y_pred != y_test)))
print("Model Accuray: {:.4f}".format(accuracy_score(y_test, y_pred)))

#Mislabeled points (RF) : 4782 out of 36723
Model Accuray: 0.8698


In [37]:
import pickle
filename = 'rfc.sav'
pickle.dump(rfc, open(filename, 'wb'))

In [ ]:
importances = rf.feature_importances_
print ("Sorted Feature Importance:")
sorted_feature_importance = sorted(zip(importances, list(X_train)), reverse=True)
print (sorted_feature_importance)

Neural Networks

Neural Network

In [ ]:
#data_train = pd.read_csv("data_train", delimiter="\t", low_memory=False)

In [ ]:
#data_train = data_train.set_index('ID RSSD')

In [ ]:
data_train.head()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models       import Sequential, load_model
from tensorflow.keras.layers       import Dense,Dropout
from tensorflow.keras.callbacks    import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.utils        import plot_model

In [ ]:
msk = np.random.rand(len(data_train)) < 0.8
train_set = data_train[msk]
test_set = data_train[~msk]

In [ ]:
X_train = train_set.drop("round_winner", axis=1)
X_test = test_set.drop("round_winner", axis=1)
y_train = train_set["round_winner"]
y_test = test_set["round_winner"]

In [ ]:
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [ ]:
X_train = np.expand_dims(X_train, 2)

In [ ]:
# Import `Sequential` from `keras.models`
from tensorflow.keras.models import Sequential

# Import `Dense` from `keras.layers`
from tensorflow.keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(20, 1)))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train, epochs=80, batch_size= 10, verbose=1)

In [ ]:
train_acc = model.evaluate(X_train, y_train, verbose=1)
test_acc = model.evaluate(X_test, y_test, verbose=1)
print(train_acc)
print(test_acc)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(3, True, 1)
for train, test in kfold.split(data):
	print('train: %s, test: %s' % (data[train], data[test]))

In [ ]:
score = model.evaluate(X_test, y_test,verbose=1)

print(score)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
from keras import optimizers

model = Sequential()
model.add(Dense(12, kernel_initializer='uniform', input_shape=(20,)))
model.add(Activation('softmax'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
from keras.optimizers import RMSprop
rmsprop = RMSprop(lr=0.0001)
model.compile(optimizer=rmsprop, loss='mse', metrics=['mae'])

In [ ]:
from keras.layers import Activation, Dense

In [ ]:
from keras import optimizers

model = Sequential()
model.add(Dense(64, kernel_initializer='uniform', input_shape=(10,)))
model.add(Activation('softmax'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(20,)))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=20, batch_size=1, verbose=1)